# Experimentos: Baseline com SVM Clássico

Este notebook estabelece baselines de performance com um modelo de Support Vector Machine (SVM) clássico em datasets

# Dataset = Breast Cancer Wisconsin (Diagnostic)

In [177]:
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC  # Importando o classificador SVM
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

## Loading data and initial analysis

In [178]:
X, y = load_breast_cancer(return_X_y=True)  # Carregando o dataset

# Verificando as dimensões dos nossos dados
print(f"Dimensões dos dados: {X.shape}")


# Vendo a distribuição das classes (0 e 1)
print("\nDistribuição das classes:")
# Usando numpy para contar as ocorrências de cada classe
unique, counts = np.unique(y, return_counts=True)
for class_label, count in zip(unique, counts):
    print(f"Classe {class_label}: {count} ocorrências")


Dimensões dos dados: (569, 30)

Distribuição das classes:
Classe 0: 212 ocorrências
Classe 1: 357 ocorrências


## Split data into training and testing sets

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify -> mantém a proporção das classes no split

print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (426, 30)
Dimensões do conjunto de teste: (143, 30)


## Normalize data

In [180]:
# normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Ajusta o scaler e transforma os dados de treino
X_test = scaler.transform(X_test)  # Transforma os dados de teste com o mesmo scaler

print("\nDados normalizados")






Dados normalizados


# Training SVM model

In [181]:
# Treinando o modelo SVM
model = SVC(kernel="linear", gamma="scale", random_state=42)  # Usando o kernel RBF
model.fit(X_train, y_train)

print("\nModelo treinado com SVM (kernel linear)")


Modelo treinado com SVM (kernel linear)


# Predictions

In [182]:
# predicting the test set
predictions = model.predict(X_test) # Prevendo os rótulos do conjunto de teste
print("\nPrevisões feitas no conjunto de teste")
# Avaliando o modelo
print("\nRelatório de Classificação:")
print(classification_report(y_test, predictions))


Previsões feitas no conjunto de teste

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        53
           1       0.99      0.99      0.99        90

    accuracy                           0.99       143
   macro avg       0.99      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143



# Dataset = Student Performance

In [183]:
# Importando o dataset
student_performance = 'student-por.csv'

df_student_performance = pd.read_csv(student_performance, sep=',')  # Lendo o dataset com separador ','

# 3. Replicamos a separação que a biblioteca fazia para nós:
# y são as colunas de notas
y = df_student_performance[['G1', 'G2', 'G3']]
# X são TODAS as outras colunas
X = df_student_performance.drop(columns=['G1', 'G2', 'G3'])

# --- Verificações para confirmar que deu tudo certo ---
print("--- Dataset carregado localmente com sucesso! ---")
print("Formato das features (X):", X.shape)
print("Formato dos alvos (y):", y.shape)

print("\n5 primeiras linhas das features (X):")


--- Dataset carregado localmente com sucesso! ---
Formato das features (X): (649, 30)
Formato dos alvos (y): (649, 3)

5 primeiras linhas das features (X):


In [184]:
X.head(5) 

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0


In [185]:
# Dimensões dos dados do Student Performance Dataset
print("\nDimensões do Student Performance Dataset:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")



Dimensões do Student Performance Dataset:
X: (649, 30)
y: (649, 3)


In [186]:
# Data cleaning
# check for missing values
print("\nVerificando valores ausentes:")
print(X.isnull().sum())


Verificando valores ausentes:
school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
dtype: int64


## Pré-processamento dataset Student Performance

In [187]:
# creating classification targets
y_final = np.where(y['G3'] >= 10, 1, 0)  # Convertendo a nota final em uma classificação binária (aprovado/reprovado). Usando 10 como o limite de aprovação.
# 1 representa aprovado e 0 representa reprovado.
# Verificando a distribuição das classes
print("\nDistribuição das classes (Aprovado/Reprovado):")
print(pd.Series(y_final).value_counts()) # Contando as ocorrências de cada classe. usando pd.Series para criar uma série pandas a partir do array numpy





Distribuição das classes (Aprovado/Reprovado):
1    549
0    100
Name: count, dtype: int64


In [188]:
# The dataset seem unbalanced, so it has to be balanced

# tranforming text features into numerical features
X_final = pd.get_dummies(X, drop_first=True)  # Convertendo variáveis categóricas em variáveis dummy # drop_first=True evita a armadilha da variável fictícia
# Verificando as dimensões dos dados após a codificação
print("\nDimensões dos dados após a codificação:") 
print("Formato das features (X) antes do processamento:", X.shape)
print("Formato das features (X) após o processamento:", X_final.shape)
# Verificando as primeiras linhas dos dados codificados
print("\nPrimeiras linhas dos dados codificados:")
print(X_final.head())
print(X_final.dtypes)




Dimensões dos dados após a codificação:
Formato das features (X) antes do processamento: (649, 30)
Formato das features (X) após o processamento: (649, 39)

Primeiras linhas dos dados codificados:
   age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  goout  \
0   18     4     4           2          2         0       4         3      4   
1   17     1     1           1          2         0       5         3      3   
2   15     1     1           1          2         0       4         3      2   
3   15     4     2           1          3         0       3         2      2   
4   16     3     3           1          2         0       4         3      2   

   Dalc  ...  guardian_mother  guardian_other  schoolsup_yes  famsup_yes  \
0     1  ...             True           False           True       False   
1     1  ...            False           False          False        True   
2     2  ...             True           False           True       False   
3     1  ...     

## Spliting data into training and testing sets

In [189]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42, stratify=y_final)  # Stratify -> mantém a proporção das classes no split

print(f"\nDimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (486, 39)
Dimensões do conjunto de teste: (163, 39)


## Normalizing 

In [190]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)  

## Using SMOTE to balance

In [191]:
# Using SMOTE to balance the dataset
from imblearn.over_sampling import SMOTE

print("Distribuição de classes no treino ANTES do SMOTE:")
print(pd.Series(y_train).value_counts())

# aplying SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Verificando o balanceamento DEPOIS do SMOTE
print("\nDistribuição de classes no treino DEPOIS do SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Distribuição de classes no treino ANTES do SMOTE:
1    411
0     75
Name: count, dtype: int64

Distribuição de classes no treino DEPOIS do SMOTE:
1    411
0    411
Name: count, dtype: int64


## Training the dataset

In [192]:
# training the SVM model with the balanced dataset

svm_model = SVC(kernel='rbf', gamma='scale', random_state=42)


svm_model.fit(X_train_resampled, y_train_resampled)

print("Modelo treinado com sucesso!")

# Predicting the test set with the SVM model
predictions = svm_model.predict(X_test_scaled)

print("\n--- Relatório de Classificação para o SVM Clássico (Student Performance) ---")
print(classification_report(y_test, predictions))

Modelo treinado com sucesso!

--- Relatório de Classificação para o SVM Clássico (Student Performance) ---
              precision    recall  f1-score   support

           0       0.36      0.40      0.38        25
           1       0.89      0.87      0.88       138

    accuracy                           0.80       163
   macro avg       0.62      0.63      0.63       163
weighted avg       0.81      0.80      0.80       163

